In [216]:
import numpy as np
import pandas as pd
import sympy as sym
from sympy import symbols, simplify, trigsimp, expand
from matplotlib import pyplot as plt


from Calculation.geometry_util import pol2cart, cart2pol, r_ort, tet_ort, phi_ort
from Calculation.material import StaticMaterial, Material, LayeredFilm, Hk2KuJ, print_Hext_parameters

In [217]:
H = symbols("H", real=True, positive=True)
Htet, Hphi = symbols("theta_H phi_H", real=True)
Ms = symbols("Ms", real=True, positive=True)
Mtet = symbols("theta_M1 theta_M2", real=True)
Mphi = symbols("phi_M1 phi_M2", real=True)
A, Ku2, gamma, alpha = symbols("A K_u2 gamma alpha", real=True, positive=True)
J = symbols("J", real=True)
Hk1, Hk2, H2k = symbols("H_k1 H_k2 H2_k", real=True, positive=True)
t = symbols("t", real=True, positive=True)

In [218]:
Ev1 = -Ms * H * (sym.sin(Htet) * sym.sin(Mtet[0]) * sym.cos(Hphi - Mphi[0]) +
                 sym.cos(Htet) * sym.cos(Mtet[0])) + \
      A * (sym.sin(Mtet[0]) ** 2) + \
      Ku2 * (sym.sin(Mtet[0]) ** 4)

Ev2 = -Ms * H * (sym.sin(Htet) * sym.sin(Mtet[1]) * sym.cos(Hphi - Mphi[1]) +
                 sym.cos(Htet) * sym.cos(Mtet[1])) + \
      A * (sym.sin(Mtet[1]) ** 2) + \
      Ku2 * (sym.sin(Mtet[1]) ** 4)

Es = t * Ev1 + t * Ev2 + \
     J * (sym.sin(Mtet[0]) * sym.sin(Mtet[1]) * sym.cos(Mphi[1] - Mphi[0]) +
          sym.cos(Mtet[0]) * sym.cos(Mtet[1]))

In [235]:
sys_mat = np.zeros((4,4), dtype=sym.Add)

sys_vars = [Mtet[0], Mphi[0], Mtet[1], Mphi[1]]
diff_vars = [Mphi[0], Mtet[0], Mphi[1], Mtet[1]]
for i in range(4):
    for j in range(4):
        sys_mat[i][j] = (gamma / Ms) * (1 / t) * Es.diff(diff_vars[i], sys_vars[j])
        if i%2==1:
            sys_mat[i][j] = -sys_mat[i][j]

lamb = symbols("lambda", complex=True)
w_sym = symbols("w", real=True)

for i in range(2):
    sys_mat[i][i] += lamb*sym.sin(Mtet[0])
    
for i in range(2, 4):
    sys_mat[i][i] += lamb*sym.sin(Mtet[1])

sys_mat = sym.Matrix(sys_mat)

# subs for PMA
sys_mat = sys_mat.subs({
    Hphi: 0, Mphi[0]: 0, Mphi[1]: 0
})
#subs for Hks
sys_mat = sys_mat.subs({
    Ku2: Ms * Hk2 / 4,
    A: Ms * (Hk1 - Hk2) / 2,
    J: (t / 2) * Ms * (H2k - Hk1)
})
#subs fow w
sys_mat = sys_mat.subs({
    lamb: sym.I*w_sym
})
#subs for our H-in-plane problem
sys_mat = sys_mat.subs({
    Htet: sym.pi/2,
    Mtet[1]: -Mtet[0]#,
    #H2k: Hk1
})

sys_mat.simplify()

In [236]:
sys_mat

Matrix([
[                                                                                                                                             I*w*sin(theta_M1), gamma*(2*H + (H2_k - H_k1)*sin(theta_M1))*sin(theta_M1)/2,                                                                                                                                                             0,                    gamma*(-H2_k + H_k1)*sin(theta_M1)**2/2],
[gamma*(-2*H*sin(theta_M1) - 2*H2_k*sin(theta_M1)**2 + H2_k + 6*H_k1*sin(theta_M1)**2 - 3*H_k1 + 8*H_k2*sin(theta_M1)**4 - 10*H_k2*sin(theta_M1)**2 + 2*H_k2)/2,                                         I*w*sin(theta_M1),                                                                                                                        -gamma*(H2_k - H_k1)*cos(2*theta_M1)/2,                                                          0],
[                                                                                                              

In [237]:
sys_det = sys_mat.det(method='berkowitz')

In [238]:
sys_det_collected = sym.collect(sym.expand(sys_det), w_sym)
pows = []
for i in range(5):
    pows.append(sys_det_collected.coeff(w_sym, i))

In [240]:
p4 = pows[4]
p4

sin(theta_M1)**4

In [242]:
pows[3]

0

In [243]:
p2 = sym.collect(pows[2], gamma)
p2 = sym.expand(sym.expand_trig(p2))
p2 = simplify(sym.trigsimp(sym.factor(p2)))
p2

gamma**2*(-2*H**2 + 2*H2_k*H_k1*sin(theta_M1)**2 - H2_k*H_k1 + 4*H2_k*H_k2*sin(theta_M1)**4 - 5*H2_k*H_k2*sin(theta_M1)**2 + H2_k*H_k2 - 2*H_k1**2*sin(theta_M1)**2 + H_k1**2 - 4*H_k1*H_k2*sin(theta_M1)**4 + 5*H_k1*H_k2*sin(theta_M1)**2 - H_k1*H_k2)*sin(theta_M1)**4

In [244]:
pows[1]

0

In [247]:
p0 = sym.collect(pows[0], [gamma])
p0 = sym.expand(sym.expand_trig(p0))
p0 = sym.trigsimp(sym.factor(p0))
p0

H**2*gamma**4*(H**2*sin(theta_M1)**2 + 4*H2_k*H_k1*sin(theta_M1)**4 - 4*H2_k*H_k1*sin(theta_M1)**2 + H2_k*H_k1 + 8*H2_k*H_k2*sin(theta_M1)**6 - 14*H2_k*H_k2*sin(theta_M1)**4 + 7*H2_k*H_k2*sin(theta_M1)**2 - H2_k*H_k2 - 8*H_k1**2*sin(theta_M1)**4 + 8*H_k1**2*sin(theta_M1)**2 - 2*H_k1**2 - 24*H_k1*H_k2*sin(theta_M1)**6 + 42*H_k1*H_k2*sin(theta_M1)**4 - 21*H_k1*H_k2*sin(theta_M1)**2 + 3*H_k1*H_k2 - 16*H_k2**2*sin(theta_M1)**8 + 40*H_k2**2*sin(theta_M1)**6 - 33*H_k2**2*sin(theta_M1)**4 + 10*H_k2**2*sin(theta_M1)**2 - H_k2**2)*sin(theta_M1)**2

In [248]:
p0_sym, p1_sym, p2_sym, p3_sym, p4_sym = symbols("p_0 p_1 p_2 p_3 p_4", real=True)
p_syms = [p0_sym, p1_sym, p2_sym, p3_sym, p4_sym]

w_sol = sym.solve(p4_sym*(w_sym**4) - p2_sym*(w_sym**2) + p0_sym, w_sym)

for i in range(4):
    w_sol[i] = w_sol[i].subs({
        p4_sym: p4,
        p2_sym: p2,
        p0_sym: p0
    })

In [251]:
w_sol[1]

sqrt(2)*sqrt(gamma**2*(-2*H**2 + 2*H2_k*H_k1*sin(theta_M1)**2 - H2_k*H_k1 + 4*H2_k*H_k2*sin(theta_M1)**4 - 5*H2_k*H_k2*sin(theta_M1)**2 + H2_k*H_k2 - 2*H_k1**2*sin(theta_M1)**2 + H_k1**2 - 4*H_k1*H_k2*sin(theta_M1)**4 + 5*H_k1*H_k2*sin(theta_M1)**2 - H_k1*H_k2) - sqrt(-4*H**2*gamma**4*(H**2*sin(theta_M1)**2 + 4*H2_k*H_k1*sin(theta_M1)**4 - 4*H2_k*H_k1*sin(theta_M1)**2 + H2_k*H_k1 + 8*H2_k*H_k2*sin(theta_M1)**6 - 14*H2_k*H_k2*sin(theta_M1)**4 + 7*H2_k*H_k2*sin(theta_M1)**2 - H2_k*H_k2 - 8*H_k1**2*sin(theta_M1)**4 + 8*H_k1**2*sin(theta_M1)**2 - 2*H_k1**2 - 24*H_k1*H_k2*sin(theta_M1)**6 + 42*H_k1*H_k2*sin(theta_M1)**4 - 21*H_k1*H_k2*sin(theta_M1)**2 + 3*H_k1*H_k2 - 16*H_k2**2*sin(theta_M1)**8 + 40*H_k2**2*sin(theta_M1)**6 - 33*H_k2**2*sin(theta_M1)**4 + 10*H_k2**2*sin(theta_M1)**2 - H_k2**2)*sin(theta_M1)**6 + gamma**4*(-2*H**2 + 2*H2_k*H_k1*sin(theta_M1)**2 - H2_k*H_k1 + 4*H2_k*H_k2*sin(theta_M1)**4 - 5*H2_k*H_k2*sin(theta_M1)**2 + H2_k*H_k2 - 2*H_k1**2*sin(theta_M1)**2 + H_k1**2 - 4*H_k

In [258]:
st_state_res = sym.solve((sym.sin(Mtet[0])**3)*4*Ku2+sym.sin(Mtet[0])*2*(A+J)-H*Ms, sym.sin(Mtet[0]), cubic=True, exclude=(Ku2,A,J,H,Ms))

In [259]:
st_state_res[2]

(H*Ms/(8*K_u2) + sqrt(H**2*Ms**2/(64*K_u2**2) + (A + J)**3/(216*K_u2**3)))**(1/3) - (A + J)/(6*K_u2*(H*Ms/(8*K_u2) + sqrt(H**2*Ms**2/(64*K_u2**2) + (A + J)**3/(216*K_u2**3)))**(1/3))

In [265]:
sym.radsimp(w_sol[1].subs({sym.sin(Mtet[0]): st_state_res[2]}))

sqrt(2)*sqrt(-(648*H**2*gamma**2*(A/(K_u2*(H*Ms/(8*K_u2) + sqrt(A**3/(216*K_u2**3) + A**2*J/(72*K_u2**3) + A*J**2/(72*K_u2**3) + H**2*Ms**2/(64*K_u2**2) + J**3/(216*K_u2**3)))**(1/3)) + J/(K_u2*(H*Ms/(8*K_u2) + sqrt(A**3/(216*K_u2**3) + A**2*J/(72*K_u2**3) + A*J**2/(72*K_u2**3) + H**2*Ms**2/(64*K_u2**2) + J**3/(216*K_u2**3)))**(1/3)) - 6*(H*Ms/(8*K_u2) + sqrt(A**3/(216*K_u2**3) + A**2*J/(72*K_u2**3) + A*J**2/(72*K_u2**3) + H**2*Ms**2/(64*K_u2**2) + J**3/(216*K_u2**3)))**(1/3))**4 - 18*H2_k*H_k1*gamma**2*(A/(K_u2*(H*Ms/(8*K_u2) + sqrt(H**2*Ms**2/(64*K_u2**2) + (A + J)**3/(216*K_u2**3)))**(1/3)) + J/(K_u2*(H*Ms/(8*K_u2) + sqrt(H**2*Ms**2/(64*K_u2**2) + (A + J)**3/(216*K_u2**3)))**(1/3)) - 6*(H*Ms/(8*K_u2) + sqrt(H**2*Ms**2/(64*K_u2**2) + (A + J)**3/(216*K_u2**3)))**(1/3))**2*(A/(K_u2*(H*Ms/(8*K_u2) + sqrt(A**3/(216*K_u2**3) + A**2*J/(72*K_u2**3) + A*J**2/(72*K_u2**3) + H**2*Ms**2/(64*K_u2**2) + J**3/(216*K_u2**3)))**(1/3)) + J/(K_u2*(H*Ms/(8*K_u2) + sqrt(A**3/(216*K_u2**3) + A**2*J/(72*K

In [252]:
tet1, phi1, tet2, phi2 = symbols("tet1 phi1 tet2 phi2")
#to_solve = sys_mat_augment.subs({
#    Hk1: 7.5e3,
#    Hk2: 6.5e3,
#    H2k: 12.5e3,
#    t: 12e-7,
#    Ms: 620
#})
res = sym.linsolve((sys_mat, sym.Matrix([[0],[0],[0],[0]])), [tet1, phi1, tet2, phi2])
res

KeyboardInterrupt: 

In [257]:
st_state_res[2]

(H*Ms/(8*K_u2) + sqrt(H**2*Ms**2/(64*K_u2**2) + (A + J)**3/(216*K_u2**3)))**(1/3) - (A + J)/(6*K_u2*(H*Ms/(8*K_u2) + sqrt(H**2*Ms**2/(64*K_u2**2) + (A + J)**3/(216*K_u2**3)))**(1/3))

In [234]:
mat_to_solve = sys_mat.subs({
    Htet: sym.pi/2,
    Mtet[1]: -Mtet[0],
    H2k: Hk1
})

eqs = []
sys_vars = [tet1, phi1, tet2, phi2]
for i in range(2):
    re_eq = sym.Rational(0,1)
    im_eq = sym.Rational(0,1)
    for j in range(2):
        re_eq += mat_to_solve[i, j]*sys_vars[j]
        #im_eq += sym.im(mat_to_solve[i, j]*sys_vars[j])
    eqs.append(re_eq)
    #eqs.append(im_eq)


res = sym.solve(eqs, [tet1, phi1])
eqs

[H*gamma*phi1*sin(theta_M1) + I*tet1*w*sin(theta_M1),
 gamma*tet1*(-2*H*sin(theta_M1) + 4*H_k1*sin(theta_M1)**2 - 2*H_k1 + 8*H_k2*sin(theta_M1)**4 - 10*H_k2*sin(theta_M1)**2 + 2*H_k2)/2 + I*phi1*w*sin(theta_M1)]

In [231]:
mat_to_solve

Matrix([
[                                                                                                            I*w*sin(theta_M1), H*gamma*sin(theta_M1),                                                                                                                            0,                      0],
[gamma*(-2*H*sin(theta_M1) + 4*H_k1*sin(theta_M1)**2 - 2*H_k1 + 8*H_k2*sin(theta_M1)**4 - 10*H_k2*sin(theta_M1)**2 + 2*H_k2)/2,     I*w*sin(theta_M1),                                                                                                                            0,                      0],
[                                                                                                                            0,                     0,                                                                                                           -I*w*sin(theta_M1), -H*gamma*sin(theta_M1)],
[                                                                                    

In [183]:
res[0][tet1]

-w*sin(theta_M1)*im(phi1)/(H*gamma*cos(theta_M1) - 2*H_k1*gamma*sin(theta_M1)**2 + H_k1*gamma - 4*H_k2*gamma*sin(theta_M1)**4 + 5*H_k2*gamma*sin(theta_M1)**2 - H_k2*gamma)

In [184]:
res[0][phi1]

re(phi1) + I*im(phi1)

In [179]:
mat_to_solve

Matrix([
[                                                                                                            I*w*sin(theta_M1),                 0,                                                                                                                             0,                  0],
[gamma*(-2*H*cos(theta_M1) + 4*H_k1*sin(theta_M1)**2 - 2*H_k1 + 8*H_k2*sin(theta_M1)**4 - 10*H_k2*sin(theta_M1)**2 + 2*H_k2)/2, I*w*sin(theta_M1),                                                                                                                             0,                  0],
[                                                                                                                            0,                 0,                                                                                                            -I*w*sin(theta_M1),                  0],
[                                                                                                         

In [173]:
mat_to_solve = mat_to_solve.subs({
    Htet: 0,
    Mtet[1]: -Mtet[0]#,
    #H2k: Hk1
})


Matrix([
[                                                                                                                            0, 0,                                                                                                                             0, 0],
[gamma*(-2*H*cos(theta_M1) + 4*H_k1*sin(theta_M1)**2 - 2*H_k1 + 8*H_k2*sin(theta_M1)**4 - 10*H_k2*sin(theta_M1)**2 + 2*H_k2)/2, 0,                                                                                                                             0, 0],
[                                                                                                                            0, 0,                                                                                                                             0, 0],
[                                                                                                                            0, 0, gamma*(-2*H*cos(theta_M1) + 4*H_k1*sin(theta_M1)**2 - 2*H_k1 + 8*H_k2*sin(

In [160]:
res[0].keys()

dict_keys([re(a), im(a), re(b), im(b), re(d), im(d), re(c), im(c)])

In [166]:
res[0][sym.re(a)]

a

In [167]:
res[0][sym.im(a)]

0

In [164]:
res[0][sym.re(b)]

a*(I*H*gamma*cos(theta_M1)/(w*sin(theta_M1)) - 2*I*H_k1*gamma*sin(theta_M1)/w + I*H_k1*gamma/(w*sin(theta_M1)) - 4*I*H_k2*gamma*sin(theta_M1)**3/w + 5*I*H_k2*gamma*sin(theta_M1)/w - I*H_k2*gamma/(w*sin(theta_M1))) + b

In [165]:
res[0][sym.im(b)]

a*(-H*gamma*cos(theta_M1)/(w*sin(theta_M1)) + 2*H_k1*gamma*sin(theta_M1)/w - H_k1*gamma/(w*sin(theta_M1)) + 4*H_k2*gamma*sin(theta_M1)**3/w - 5*H_k2*gamma*sin(theta_M1)/w + H_k2*gamma/(w*sin(theta_M1)))

In [168]:
res[0][sym.re(d)]

c*(-I*H*gamma*cos(theta_M1)/(w*sin(theta_M1)) + 2*I*H_k1*gamma*sin(theta_M1)/w - I*H_k1*gamma/(w*sin(theta_M1)) + 4*I*H_k2*gamma*sin(theta_M1)**3/w - 5*I*H_k2*gamma*sin(theta_M1)/w + I*H_k2*gamma/(w*sin(theta_M1))) + d

In [170]:
res[0][sym.im(d)]

c*(H*gamma*cos(theta_M1)/(w*sin(theta_M1)) - 2*H_k1*gamma*sin(theta_M1)/w + H_k1*gamma/(w*sin(theta_M1)) - 4*H_k2*gamma*sin(theta_M1)**3/w + 5*H_k2*gamma*sin(theta_M1)/w - H_k2*gamma/(w*sin(theta_M1)))